# Backtest a  Strategy

In [2]:
%matplotlib inline
import quantstats as qs
# extend pandas functionality with metrics, etc.
qs.extend_pandas()
from smartbots.crypto.historical_downloader import historical_downloader, save_test_data
import datetime as dt
import pandas as pd
from smartbots.engine.data_reader import read_data_to_dataframe
from smartbots.engine.portfolio_constructor import Portfolio_Constructor

## 1) Download Historical Data from Data Provider and save into the database.
Please see here the docs of the exchanges available: https://github.com/ccxt/ccxt/wiki/Manual#markets


If you already have the data in a database, you can skip this step and go to the next.

Dependings of the symbols and the length of the data, the data loader will take some time to complete. Be pacience.

If there are already some data in the database, it will update only the new data.


In [3]:
# Parameters
test = False  # set False if you have the keys for the exchange and you copy it on docker/compose.env.

if test:
    provider ='test'
    save_test_data()
else:
    provider = 'kucoin' # change for your exchange
    symbols = ["BTC-USDT","ETH-USDT"] # List of symbols to download from provider
    start_date = dt.datetime(2022,9,1) # Start date of data to download
    end_date = dt.datetime.utcnow() # End date of data to download
    interval = '1m' # Interval of data to download, 
    historical_downloader(symbols=symbols,start_date=start_date, end_date=dt.datetime.utcnow(),
           provider=provider,clean_symbols_database=[],interval=interval)

Pandas dataframe BTC-USDT_202210 contains Objects, saving as Blob


BTC-USDT Since: 2022-10-06T16:29:00.000Z
Symbol BTC-USDT saved.
* Historical data for BTC-USDT saved


Pandas dataframe ETH-USDT_202210 contains Objects, saving as Blob


ETH-USDT Since: 2022-10-06T16:29:00.000Z
Symbol ETH-USDT saved.
* Historical data for ETH-USDT saved


## 2) Read Historical Data from Database and Visualize it.


In [ ]:
provider = 'kucoin'
symbols = ["BTC-USDT","ETH-USDT"] # List of symbols to download from provider
start_date = dt.datetime(2022,8,1) # Start date of data to download
end_date = dt.datetime.utcnow() # End date of data to download
df1 = read_data_to_dataframe(symbols[0], provider,start_date=start_date,end_date=end_date)
df2 = read_data_to_dataframe(symbols[-1], provider,start_date=start_date,end_date=end_date)
df = pd.concat([df1,df2],axis=0)

In [ ]:
df1.close.plot()

## 3) BackTesting

### Settings

In [5]:
provider = 'kucoin'
inicial_cash = 50000 # initial capital for the portfolio
start_date = dt.datetime(2022,8,1) # Start date for backtesting


strategy1 = {'id': 1, # unique id for strategy to run
             'strategy': 'Simple_Avg_Cross', # strategy name, create in smartbots/crypto/strategies
             'params': {'ticker': 'BTC-USDT',
             'short_period': 10,
             'long_period': 500,
             'quantity': 1,
             'limit_save_values':1000,'point_value':1,'slippage':0}}

strategy2 = {'id': 2, # unique id for strategy to run
             'strategy': 'Simple_Avg_Cross', # strategy name, create in smartbots/crypto/strategies
             'params': {'ticker': 'ETH-USDT',
             'short_period': 10,
             'long_period': 500,
             'quantity': 1,
             'limit_save_values':1000,'point_value':1,'slippage':0}}



conf_portfolio = {'Name': 'PortfolioCrypto1',
                  'Data_Sources': 
                  [{'tickers': ['BTC-USDT','ETH-USDT'],
                   'event_type': 'bar',
                   'historical_source': 'mongoDB',
                   'historical_library': f'{provider}_historical_1min'}],
                   'Strategies': 
                    [strategy1, strategy2]}

In [ ]:
portfolio = Portfolio_Constructor(conf_portfolio, run_real=False, asset_type='crypto',
                                             send_orders_to_broker=False, start_date=start_date,
                                  inicial_cash=inicial_cash)
portfolio.run_simulation() # backtest 

info_port = portfolio.get_saved_values_strategy() # read change of positions

BTC-USDT_202208
ETH-USDT_202208


## Statistics

Plots and statistics of the Strategies and the Portfolio.

In [ ]:
equities =portfolio.equity_handler.get_equities()
portfolio = equities['equity_portfolio']['equity']
port_return = portfolio.pct_change()
qs.plots.snapshot(portfolio, title='Portfolio Performance')

In [ ]:
qs.reports.metrics(port_return, "BTC",mode= 'basic')